In [1]:
# Data Preprocessing
import pandas as pd
import json
import nltk
import numpy as np
import sqlite3

# Generative Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import RNN
from keras.utils import np_utils

Using TensorFlow backend.


# Got Instagram Data From Data Privacy Download

In [2]:
allComments = []

def parseComment(body):
    temp = []
    for comment in body:
        tokens = nltk.word_tokenize(comment[1])
        if '@' in tokens:
            index = tokens.index('@')
            del tokens[index:index+2]
        comment = ' '.join(tokens)
        if comment != '':
            temp.append(comment.lower())
    return temp

with open('./h93978/comments.json') as f:
    h93978Comments = json.load(f)['media_comments']
with open('./dalyhayden/comments.json') as f:
    dalyhaydenComments = json.load(f)['media_comments']
h93978Comments = parseComment(h93978Comments)
dalyhaydenComments = parseComment(dalyhaydenComments)

In [3]:
allMessages = []

def parseMessage(body):
    temp = []
    for conversation in body:
        for message in conversation['conversation']:
            if message['sender'] == 'h93978' or message['sender'] == 'dalyhayden':
                if 'text' in message:
                    tokens = nltk.word_tokenize(message['text'])
                    if '@' in tokens:
                        index = tokens.index('@')
                        del tokens[index:index+2]
                    message = ' '.join(tokens)
                    if message != '':
                        temp.append(message.lower())
    return temp

with open('./h93978/messages.json') as f:
    h93978Messages = json.load(f)
with open('./dalyhayden/messages.json') as f:
    dalyhaydenMessages = json.load(f)
h93978Messages = parseMessage(h93978Messages)
dalyhaydenMessages = parseMessage(dalyhaydenMessages)        

In [4]:
file = open("personalData.txt", "a")
for row in h93978Messages: 
    file.write(row + "\n")
for row in h93978Comments:
    file.write(row + "\n")
for row in dalyhaydenMessages:
    file.write(row + "\n")
for row in dalyhaydenComments:
    file.write(row + "\n")
file.close()

# Got Message Data from iPhone Backup
In /Library/Application Support/MobileSync/3d/
The file was called '3d0d7e5fb2ce288813306e4d4636395e047a3d28' and then I added the extension '.mddata' and opened it in SQLLiteStudio.

In [5]:
def create_connection(path):
    connection = None
    connection = sqlite3.connect(path)
    print("Connection to SQLite DB successful")
    return connection

In [6]:
connection = create_connection('./iphone/iphonedata.mddata')

Connection to SQLite DB successful


In [7]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    cursor.execute(query)
    result = cursor.fetchall()
    return result

In [8]:
select_messages = "SELECT text FROM message WHERE is_from_me = 1"
messages = execute_read_query(connection, select_messages)
file = open("personalData.txt", "a")
for message in messages:
    if message[0] != None:
        if 'https' not in message[0]:
            file.write(message[0] + "\n")
file.close()

# Simple Text Generation Model;

In [9]:
text = (open("./personalData.txt").read())
text = text.lower()
characters = sorted(list(set(text)))
n_to_char = { n:char for n, char in enumerate(characters)}
char_to_n = { char:n for n, char in enumerate(characters)}

In [10]:
X = []
Y = []
length = len(text)
seq_length = 100

for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label =text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

In [11]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [12]:
model = Sequential()
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [13]:
model.fit(X_modified, Y_modifipip insted, epochs=1, batch_size=100)
model.save_weights('./models/baseline.h5')

Epoch 1/1
859976/859976 [==============================] - 11746s 14ms/step - loss: 2.7336


OSError: Unable to create file (unable to open file: name = './models/baseline.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 602)

In [ ]:
model.load_weights('./models/baseline.h5')

In [ ]:
string_mapped = X[99]
full_string = [n_to_char[value] for value in string_mapped]
# generating characters
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

In [ ]:
txt=""
for char in full_string:
    txt = txt+char
txt